STEM post processing widget

Stephanie Ribet

June 2023

TODO: re run simulations with frozen phonons

Data: [3D array](https://drive.google.com/file/d/1MtO_pYm-d4gZ__XQTF3p--RK2_HqkGM2/view?usp=sharing), 
[DPC](https://drive.google.com/file/d/1Mt8KOIxw4u5I7PvbQwjPDrevf8JJZVCP/view?usp=sharing)

In [1]:
%matplotlib widget

In [2]:
import numpy as np
import matplotlib.pyplot as plt

In [3]:
from IPython.display import display
from ipywidgets import HBox, VBox, widgets, interact, Dropdown, Label
# from ipywidgets import widgets, interact, GridspecLayout, Layout,  Layout, Label, 
from matplotlib import cm
from ipywidgets import AppLayout, FloatSlider, FloatLogSlider, Layout
import h5py
from abtem.utils import fft_interpolate_2d
# from py4DSTEM.visualize import Complex2RGB

In [4]:
hf = h5py.File('STO_DPC.h5', 'r')
DPC = hf.get('DPC')[:]
hf.close()

In [5]:
hf = h5py.File('STO_3D.h5', 'r')
array_3D = hf.get('3D')[:]
hf.close()

In [6]:
array_3D = np.squeeze(array_3D)

In [7]:
qx = np.fft.fftfreq(array_3D.shape[2] * 2, 1/array_3D.shape[2]/2)
qy = np.fft.fftfreq(array_3D.shape[2] * 2, 1/array_3D.shape[2]/2)
qya, qxa = np.meshgrid(qx, qy)
qra = np.fft.fftshift(np.sqrt(qxa**2 + qya **2))

In [10]:
with plt.ioff():
    fig = plt.figure(figsize = (9,3))

ax0 = fig.add_axes([0.04,  0.05,  0.28, 0.75])
ax1 = fig.add_axes([0.37,  0.05,  0.28, 0.75])
ax2 = fig.add_axes([0.70,  0.05,  0.28, 0.75])
ax3 = fig.add_axes([0.039,  0.6,  0.1, 0.2])
ax4 = fig.add_axes([0.369,  0.6,  0.1, 0.2])

cmap = 'gray'
tile = 3

im0 = ax0.imshow(
        np.tile(array_3D[:,:,0:22].sum(2),(tile,tile)), 
    cmap = cmap)
im1 = ax1.imshow(
        np.tile(array_3D[:,:,22:100].sum(2),(tile,tile)), 
    cmap = cmap)
im2 = ax2.imshow(
    np.tile(DPC, (tile,tile)),
    cmap = cmap)

cmap2 = "YlGn_r"
im3 = ax3.imshow(
    qra < 22,
    cmap = cmap2,
    vmax = 1.5, 
    vmin = -0.5
)

im4 = ax4.imshow(
    np.logical_and(qra > 22, qra < 200),
    cmap = cmap2,
    vmax = 1.5, 
    vmin = -0.5
)

ax0.set_xticks([])  
ax0.set_yticks([]) 
ax1.set_xticks([])  
ax1.set_yticks([]) 
ax2.set_xticks([])  
ax2.set_yticks([]) 
ax3.set_xticks([])  
ax3.set_yticks([]) 
ax4.set_xticks([])  
ax4.set_yticks([]) 

ax0.set_title('Circular virtual image');
ax1.set_title('Annular virtual image');
ax2.set_title('DPC reconstruction');

def update_ims(BF_radius, DF_radius_1, DF_radius_2):
    
    BF = np.tile(array_3D[:,:,0:BF_radius].sum(2),(tile,tile))
    im0.set_data(BF)
    im0.set_clim(vmax = BF.max(), vmin = BF.min())
    DF = np.tile(array_3D[:,:,DF_radius_1:DF_radius_2].sum(2),(tile,tile))
    im1.set_data(DF)
    im1.set_clim(vmax = DF.max(), vmin = DF.min())
    im3.set_data(qra < BF_radius)
    im4.set_data(np.logical_and(qra > DF_radius_1, qra < DF_radius_2))

    fig.canvas.draw_idle()

style = {
    'description_width': 'initial',
}

BF_radius = widgets.IntSlider(
    value = 22, min = 0, max =200, 
    step = 1,
    description = "BF radius (mrad)",
    style = style
)


DF_radius_1 = widgets.IntSlider(
    value = 22, min = 0, max = 200, 
    step = 1,
    description = "DF inner radius (mrad)",
    style = style
)


DF_radius_2 = widgets.IntSlider(
    value = 100, min = 0, max = 200, 
    step = 1,
    description = "DF outer radius (mrad)",
    style = style
)

widgets.interactive_output(
    update_ims, 
    {
        'BF_radius':BF_radius,
        'DF_radius_1':DF_radius_1,
        'DF_radius_2':DF_radius_2,
    },
)



option_list = (
    'select an option',
    'BF',
    'ABF',
    'ADF',
    'HAADF',
)

# update the plots with a pre-selected function
def select_preset_eventhandler(change):
    if change.new == option_list[1]: #BF
        BF_radius.value = 22
    
    if change.new == option_list[2]: #ABF
        DF_radius_1.value = 7
        DF_radius_2.value = 20
        
    if change.new == option_list[3]: #ADF
        DF_radius_1.value = 50
        DF_radius_2.value = 120
    
    if change.new == option_list[4]: #HAADF
        DF_radius_1.value = 90
        DF_radius_2.value = 200
            
# Widgets
dropdown = Dropdown(
    options = option_list,
    layout = Layout(width='200px',height='30px'),
)
dropdown.observe(select_preset_eventhandler, names='value')

widgets.VBox(
    [
        fig.canvas,
        HBox([
            BF_radius, 
            VBox([DF_radius_1,DF_radius_2]),
            dropdown
        ]),
    ],
)


In [11]:
array_3D.shape

(32, 32, 263)